In [16]:
import pickle
import os
import pandas as pd
from glob import glob

In [18]:
# # 删除问题行
# metadata_path = "/data_nas/GoogleSV/metadata/China/HongKong/pano_2024-08-17 13:11:09.125553_23512.p"
# metadata = pd.read_pickle(metadata_path)
# error_line = metadata[metadata['panoid'] == "AF1QipNot5b97DiHOQQDAASe-iSdgapfQ3nJIcZY_QpL"]
# metadata = metadata.drop(error_line.index).reset_index(drop=True)
# metadata.to_pickle(metadata_path)

In [8]:
dataset_path = "/data_nas/cehou/LLM_safety"
dataset = os.listdir(dataset_path)

In [9]:
glob(f'{dataset_path}/dataset_30_male_HongKong_murder_*')

['/data_nas/cehou/LLM_safety/dataset_30_male_HongKong_murder_560.pkl']

In [10]:
for i,file in enumerate(glob(f'{dataset_path}/dataset_30_male_HongKong_murder_*')):
    with open(file, "rb") as f:
        data = pickle.load(f)
        print(len(data))
        if i == 0:
            all_data = data
        else:
            all_data.extend(data)


560


In [11]:
all_data[213]

{'GSV_idx': 445495,
 'GSV_name': '27K5yO2L2dNA95G-dc0_eQ',
 'GSV_path': ['/data_nas/GoogleSV/images/China/HongKong/2/7/27K5yO2L2dNA95G-dc0_eQ_0.jpg',
  '/data_nas/GoogleSV/images/China/HongKong/2/7/27K5yO2L2dNA95G-dc0_eQ_90.jpg',
  '/data_nas/GoogleSV/images/China/HongKong/2/7/27K5yO2L2dNA95G-dc0_eQ_180.jpg',
  '/data_nas/GoogleSV/images/China/HongKong/2/7/27K5yO2L2dNA95G-dc0_eQ_270.jpg'],
 'text_description': '[INST] Please design a murder-focused street safety perception system rating scale and list in as much detail as possible the different information that people pay attention to in street perception by looking around the built environment (elements of the urban environment that Street View images can capture). Note that we cannot provide subjective information about residents\' personal experiences, so look for key points from the objective environment, please answer this question within 500 words. [/INST] A murder-focused street safety perception system rating scale could be des

In [12]:
data_config = dataset[0].split("_")
save_filename = f"/data_nas/cehou/LLM_safety/dataset_{data_config[1]}_{data_config[2]}_{data_config[3]}_{data_config[4]}_{len(all_data)}.pkl"
with open(save_filename, "wb") as f:
    pickle.dump(all_data, f)
print(f"save the data into {save_filename}")

save the data into /data_nas/cehou/LLM_safety/dataset_30_male_HongKong_murder_560.pkl
